In [1]:
import argparse
import builtins
import math
import os
import random
import shutil
import time
import warnings
import numpy as np
from functools import partial

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as torchvision_models

import moco.builder
import moco.loader
import moco.optimizer

import vits

from torchinfo import summary

import matplotlib
'''
matplotlib inline
'''
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.pyplot import imshow
from PIL import Image

# input a figure

In [2]:
imgfile=r"food_bag_2_1_1_texture.png"
img = Image.open(imgfile).convert('RGB')
print(img.size)
resized_img=img.resize((224,224))
print(resized_img)
# figure(figsize=(10, 10), dpi=80)
# plt.imshow(resized_img)
convert_tensor = transforms.ToTensor()
input_img=convert_tensor(resized_img)
input_img=input_img.unsqueeze(0).to("cuda:0",torch.float)
print(input_img)

(230, 137)
<PIL.Image.Image image mode=RGB size=224x224 at 0x7F309C5BA520>
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')


# Extract MoCo encoder

In [8]:
pretrained_model_path='/home/quan/Downloads/vit-b-300ep.pth.tar'
mocomodel=vits.__dict__['vit_base']().cuda("cuda:0")
linear_keyword = 'head'
# mocomodel = moco.builder.MoCo_ViT(partial(vits.__dict__['vit_base'])).cuda("cuda:0")
checkpoint = torch.load(pretrained_model_path, map_location="cuda:0")
# rename moco pre-trained keys
state_dict = checkpoint['state_dict']
for k in list(state_dict.keys()):
    # retain only base_encoder up to before the embedding layer
    if k.startswith('module.base_encoder') and not k.startswith('module.base_encoder.%s' % linear_keyword):
        # remove prefix
        state_dict[k[len("module.base_encoder."):]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]
mocomodel.load_state_dict(state_dict, strict=False)

# remove the last classification head to identity
mocoencoder=torch.nn.Sequential(*list(mocomodel.children())[:-1])
mocomodel.head=nn.Identity()
fake_input=torch.randn((1,3,224,224)).cuda()
# print(mocoencoder)
# summary(mocoencoder,(1,3,224,224),depth=4,col_names=["kernel_size", "output_size", "num_params"])#, "mult_adds"
fake_output=mocomodel(input_img)
# m=nn.Softmax(dim=2)
fake_output=torch.squeeze(fake_output)
print(fake_output.shape)
# summary(mocomodel,(1,3,224,224),depth=4,col_names=["kernel_size", "output_size", "num_params"])#

torch.Size([768])


In [10]:
toPIL=transforms.ToPILImage()

img=toPIL(fake_output.to('cpu'))
# print(img)
# figure(figsize=(20, 10), dpi=80)
plt.imshow(img)

ValueError: pic should be 2/3 dimensional. Got 1 dimensions.